# Cost of Plutocracy

The aim of this article is to analyze some of the biggest DAOs in the crypto space and see how many of them are influenced by token holders which own a disproportionate amount of tokens compared to most voters.

In [1]:
# sets up the pynb environment
import os
import sys

import pandas as pd
import numpy as np


module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from libs.data_processing.filters import get_quartile_by_vp
from libs.data_processing.statistics import (
    get_number_of_whales_to_all_voters_ratio,
    get_score_comparisons,
)


The data used for this article came from data sources that specialize in providing data on DAO governance, such as DeepDAO and Snapshot. Coingecko was also used to find market data for DAO native tokens.

Using this data, two spreadsheets were compiled which act as the local database for this analysis. Each spreadsheet contains each voter's choice and voting power for the last five proposals each DAO opened. **One spreadsheet filters out "whales"** which, in the context of this analysis, are voters of each proposal which have voting power **at or above the 95th percentile of voting power for that proposal**.

In [2]:
plutocracy_report_data = pd.read_excel(
    "../plutocracy_report.xlsx", sheet_name=None, engine="openpyxl"
)
plutocracy_report_data_filtered = pd.read_excel(
    "../plutocracy_report_filtered.xlsx", sheet_name=None, engine="openpyxl"
)

With this, we can begin asking ourselves some questions. Such as: **How many whales have voted in the last five proposals for each of the DAOs we analyzed?**

In [3]:
voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [4]:
pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    index=[list(result.items())[0][0] for result in voting_ratios],
    columns=[
        "# of whales",
        "all voters",
        "avg vp for non-whales",
        "avg vp for whales",
        "avg cost of vote",
    ],
)


,# of whales,all voters,avg vp for non-whales,avg vp for whales,avg cost of vote
Uniswap,791,21085,1.675602,9.507515e+04,7.289716
ENS,321,9742,281.564683,5.561220e+04,29.874161
Lido,152,4384,115.907850,1.351838e+06,1.564882
Frax,2,161,112522.786545,1.436326e+07,8.416601
Curve Finance,14,235,3454.929793,1.352748e+06,3.338636
Decentraland,54,1182,251.487140,2.851501e+05,0.703804
Compound,55,1054,0.128663,5.490055e+03,47.541103
Radicle,18,395,26779.601995,9.430134e+05,12.567908


Here we can clearly see that for the DAOs which we choose to analyze the top 5% of voters have a clear, disproportionate amount of voting power comparared to the average voter.

We can also gain some insight at the economic might that these whales hold for each DAO at the time of voting by taking a look at the average cost of each vote for each DAO's native token at the time these proposals were active.

---

We move on to taking at what DAO participation would look like without these whale's involvement. We do so by comparing the scores of each proposal and checking whether or not the outcome of the proposal was changed after filtering out whales.

We check whether or not a propsal's outcome has changed simply by checking if the largest vote choice score has changed after filtering out whales, more specifically, in python we do:
```python
has_changed_outcome = not unfiltered_winning_choice_index == filtered_winning_choice_index
```

In [5]:
score_diffrences = get_score_comparisons(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [6]:
score_diffrences_dfs = dict()

for score_diffrence in score_diffrences:
    for organization, data in score_diffrence.items():
        data: dict[str, list] = data
        items = data.items()
        score_diffrences_dfs[organization] = pd.DataFrame(
            [score_data for _, score_data in items],
            index=pd.Index(
                ([proposal_id for proposal_id, _ in items]), name="Proposal ID"
            ),
            columns=[
                "type",
                "choices",
                "score_diffrences",
                "total_vp",
                "outcome_changed",
                "outcome"
            ],
        )
        score_diffrences_dfs[organization]["total_vp"] = score_diffrences_dfs[
            organization
        ]["total_vp"].apply("{:.9f}".format)
        score_diffrences_dfs[organization].index = score_diffrences_dfs[organization].index.str.slice(0, 9)


First, let's look at Uniswap

In [7]:
score_diffrences_dfs["Uniswap"].index = score_diffrences_dfs["Uniswap"].index.str.slice(0, 9)
score_diffrences_dfs["Uniswap"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0x374d7f9,single-choice,"['Yes', 'No']","[14155.085736103796, 30.84262236625821]",18625.532208988,False,Yes
0x15031b6,single-choice,"['For', 'Against', 'Abstain']","[15561440.496482328, 0.0, 0.0]",38105686.407799788,False,For
0x6b8df36,single-choice,"['Wormhole', 'LayerZero', 'Celer', 'deBridge']","[23275464.75822096, 17163633.19927637, 481.545...",45581689.082320422,False,Wormhole
0xa3c7247,single-choice,"['Yes', 'No']","[20056524.265649345, 4927056.974342918]",24994560.158377878,False,Yes
0xfd3d380,single-choice,"['add Overnight Finance', 'add DeFiEdge', 'add...","[10054.585075893452, 2708025.0728700403, 3412....",2728492.255083876,True,add Overnight Finance
0x387b5e8,single-choice,"['Yes - Enact changes', 'No - No change']","[30508464.50667036, 34.17723847808492]",61343398.710826278,False,Yes - Enact changes
0x5ad7208,single-choice,"['Yes ', 'No ']","[6025893.960343872, 3000106.0924527245]",11575893.085716439,False,Yes
0x31c3409,single-choice,"['Yes - Fix the bridge', 'No - Make no change']","[9939544.146631507, 0.0]",12950287.668343490,False,Yes - Fix the bridge
0xa4a21c8,single-choice,"['Yes', 'No']","[19833891.015727025, 0.0]",19840363.162523411,False,Yes


We can do some quick calculations using pandas to see that that 12.5% of proposals had their outcome changed by removing whales.

In [29]:
score_diffrences_dfs["Uniswap"]["outcome_changed"].value_counts(normalize=True)

False    0.875
True     0.125
Name: outcome_changed, dtype: float64

One of the proposals which had their outcome changed after removing whales was the proposal to [add a Liquidity Mining Manager for the Optimism-Uniswap Liquidty Mining Program](https://snapshot.org/#/uniswap/proposal/0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c).

In this case, "Overnight Finance" would've been the chosen LMM if it weren't for whale interevention, instead "DeFiEdge" was chosen for this campaign. From the data, we can see that without whale intervention, DeFiEdge would've had ~2,708,025 less votes out of the total ~2,728,492 voting power on this proposal. That is ~99.3% of voting power allocated for this proposal directed to choosing "DeFiEdge".

In [8]:
propsal_choices = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Uniswap"].loc["0xfd3d380"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,add Overnight Finance,add DeFiEdge,add none
Scores,14335.930887,2.710600e+06,3556.089173
Score Differeces,10054.585076,2.708025e+06,3412.851497


One highly contentious proposal whose outcome did not change after filtering out whales was [this temperature check to choose which Eth <> BNB bridge to use for Uniswap v3 governance](https://snapshot.org/#/uniswap/proposal/0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852). We see that even for the least popular choices, whales still contributed to the majority of the votes. We also observe that whales held the overwhelming majority of voting power for the runner up, "LayerZero", with 99.9% of voting power allocated to this proposal being directed to this choice. By comparison, "Wormhole" received 65.1% of voting power for this proposal coming from whales.

In [9]:
propsal_choices = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Uniswap"].loc["0x6b8df36"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,Wormhole,LayerZero,Celer,deBridge
Scores,2.840391e+07,1.717334e+07,662.606204,3772.802474
Score Differeces,2.327546e+07,1.716363e+07,481.545728,3402.436704


Next up is ENS.

In [10]:
score_diffrences_dfs["ENS"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0x41b3509,single-choice,"['For', 'Against', 'Abstain']","[1667210.963067633, 0.0, 137.12454090065714]",1670185.073618348,False,For
0xd7eff78,single-choice,"['For', 'Against', 'Abstain']","[1666938.9315321625, 100.38682608595627, 141.9...",1670154.219044510,False,For
0x5788bf0,single-choice,"['For', 'Against', 'Abstain']","[1446509.1013934365, 208348.53159207577, 13519...",1671687.088797375,False,For
0xdaff050,approval,"['nick.eth', 'james.eth', 'accessor.eth', 'sim...","[2319757.8152544936, 2319757.8152544936, 23197...",2325021.309782755,False,nick.eth
0xa245dc7,approval,"['thecap.eth (Cap)', 'slobo.eth', 'limes.eth',...","[2278346.061608189, 2278346.0616081893, 227834...",2282885.783012460,False,slobo.eth
0xa714c25,approval,"['daylon.eth', 'avsa.eth', 'vegayp.eth', 'hell...","[2294889.695027718, 2294889.6950277183, 229488...",2299996.536005338,False,avsa.eth
0x4a1aedb,basic,"['For', 'Against', 'Abstain']","[2621541.8473638818, 700.4818368034764, 0.0]",2639656.796499248,False,For
0x5c96e49,basic,"['For', 'Against', 'Abstain']","[2113673.2141601057, 0.0, 0.0]",2126140.031373791,False,For
0x9726578,basic,"['For', 'Against', 'Abstain']","[2017278.8829650308, 0.0, 326847.28692344716]",2363678.995122524,False,For


In [31]:
score_diffrences_dfs["ENS"]["outcome_changed"].value_counts(normalize=True)

False    0.964286
True     0.035714
Name: outcome_changed, dtype: float64

We see that a very small percentage of proposal outcomes were changed after filtering out whales (3.5%). However, if we take look at score differences as they compare to each score, we see that they were almost all determined by whales.

In [11]:
propsal_choices = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["ENS"].loc["0x41b3509"]["score_diffrences"]
proposal_scores = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,For,Against,Abstain
Scores,1.669865e+06,125.702762,194.817417
Score Differeces,1.667211e+06,0.000000,137.124541


[This proposal](https://snapshot.org/#/ens.eth/proposal/0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7) once passed will approve the funding of various grants ENS has commited to funding as detailed by the proposal, **the amount of capital to be commited to this proposal is 250k USDC and 50 ETH**. 99.9% of voting power allocated towards this propsal was commited to voting for its passing, 0.0001% was allocated to abstaining and even a smaller percentage was allocated to voting against its passing.

Meaning that ~0.1% of the vote would've still been allocated towards voting for the proposal to pass even without whale intervention, which I think many would see as fine since funding of public goods is always important.

Another propsal on ENS which had a large amount of voting power committed by whales is [this proposal](https://snapshot.org/#/ens.eth/proposal/0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b) to commit **935k USDC and 254 ETH** to the ENS Ecosystem Working group which is responsible for *growing and improving the ENS Ecosystem by funding people and projects that are ENS-specific or ENS-centric*.

In [12]:
propsal_choices = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["ENS"].loc["0x5788bf0"]["score_diffrences"]
proposal_scores = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,For,Against,Abstain
Scores,1.449464e+06,208582.378002,13640.353424
Score Differeces,1.446509e+06,208348.531592,13519.449465


Similar to the previous proposal we looked at, this one was highly influenced by whales (~86.5% of voting power committed to this proposal coming from whales that voted for its passing, ~12.5% from whales which voted against it, and ~0.8% of whales voted to abstain).

Yet, even after removing whales, the proposal still passes. After doing some quick math, we can see the the remaining voting power for this proposal that voted for its passing equates to about ~0.17% out of the remaining 0.2% of voting power after removing whales.

Still, I am not at all saying that the outcome of this proposal is controversial. An organisation funding initiatives that aid in the development of its product should not be a controversial topic for its members. I suppose the only thing that would be contested is the amount that is committed to this proposal.

---
Next lets take a look at everyone's favourite liquid staking platform: Lido.

In [13]:
score_diffrences_dfs["Lido"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0xc00b482,single-choice,"['For', 'Against']","[9121489.370392222, 69.40333267123832]",9124945.808187168,False,For
0x3436b98,single-choice,"['For', 'Against']","[55124323.664074376, 2048432.8880345668]",57176658.163439780,False,For
0x32f6f09,single-choice,"['For', 'Against']","[57163748.21209293, 7563.113010643607]",57175091.606986538,False,For
0x7ac2431,single-choice,"['For', 'Against']","[57686947.46411716, 78.13038422652139]",57690201.588526487,False,For
0x7f19fed,single-choice,"['YAY', 'NAY']","[54546195.85193226, 7525.505910950667]",54557517.318199627,False,YAY
0xcbf5343,single-choice,"['1. For 6 new oracles, 6/11 set', '2. For 4 n...","[92934.92264371048, 52723122.2034864, 4535.772...",52826368.429703690,True,"1. For 6 new oracles, 6/11 set"
0xbb5c7f1,single-choice,"['Upgrade to V2', 'Do not upgrade to V2']","[38446944.67494801, 8455.104714179814]",38460661.685549140,False,Upgrade to V2
0xb2732eb,single-choice,"['For', 'Against']","[31927484.633468445, 7050320.1428119475]",38982679.409992553,False,For
0xce01f9f,single-choice,"['For', 'Against']","[50564569.52830553, 10941.755267164812]",50589095.777857043,False,For


In [34]:
score_diffrences_dfs["Lido"]["outcome_changed"].value_counts(normalize=True)

False    0.96
True     0.04
Name: outcome_changed, dtype: float64

Here, we see that there was a change in outcome for just two of the proposals (4%). [This proposal](https://snapshot.org/#/lido-snapshot.eth/proposal/0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b) to [expand the oracle set and quorum](https://mainnet.lido.fi/#/lido-dao/0x442af784a788a5bd6f42a01ebe9f287a871243fb/) of Lido DAO oracle node operators was passed 16/12/22 and onboarded 4 new oracle node operators and set the quorum to 5/9 reports from orales being identical to be accepted by the protocol.

Had whales not intervened, Option 1 to onboard 6 additional oracles and set the quorum to 6/11 would've passed.

In [14]:
propsal_choices = plutocracy_report_data['Lido'][plutocracy_report_data['Lido']['proposal_id'] == '0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Lido"].loc["0xcbf5343"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Lido'][plutocracy_report_data['Lido']['proposal_id'] == '0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,"1. For 6 new oracles, 6/11 set","2. For 4 new oracles, 5/9 set",3. Against
Scores,96886.616711,5.272470e+07,4778.954352
Score Differeces,92934.922644,5.272312e+07,4535.772055


Here we see that whale participation was consistent for all choices for this proposal. In fact, ~99.982% of voting power allocated to this proposal came from whales that voted for each of the outcomes. The second outcome alone had ~99.980% of the voting power allocated to this proposal, so obviously its removal would make a huge impression on the outcome.

Indeed, the voting power would be comparitively low without whale intervention (if this were the case, only ~9277.7397 LDO would be allocated to this proposal). However the fact that after doing so additional oracles would still be implemented anyway, where the only other alternative would be to not implement any more oracles, shows that the community that participated in this proposal still saw it important to add more oracles to further promote decentralisation of oracles which [report chain state from the beacon chain to the execution layer](https://docs.lido.fi/guides/oracle-operator-manual) of ETH 2.0 as it pertains to the operation of Lido.

In conclusion of this analysis of this proposal, we can say that whales and average token holders were moving the same direction, but perhaps taking different paths. It will be interesting to see why whales opted to implement less oracles than what the majority of average token holders chose, which may require a more in depth report based on the [forum discussions](https://research.lido.fi/t/expansion-of-lidos-ethereum-oracle-set/2836) on the propoal.

---
Now we take a look at Frax.

In [15]:
score_diffrences_dfs["Frax"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0x1c844e1,weighted,"['For', 'Against']","[1432.8549342407946, 0]",2594.203582734,False,For
0x1127ca8,weighted,"['For', 'Against']","[4485.967956881548, 0.0]",7397.700806661,False,For
0x8f497b3,weighted,"['For', 'Against']","[29460.77362414489, 0.0]",33806.253531748,False,For
0x9547dee,weighted,"['For', 'Against']","[26582392.78665665, 0]",26594367.493811361,False,For
0x1272233,weighted,"['For', 'Against']","[36401823.83824842, 0.0]",41369202.344253987,False,For
0x9cc997c,weighted,"['For', 'Against']","[32596053.830188096, 0.0]",40176140.875503592,False,For
0x02aa360,weighted,"['For', 'Against']","[32596092.76074356, 0]",40116426.022636428,False,For
0xe53269e,weighted,"['For', 'Against']","[32596126.82497959, 0]",40271805.931466728,False,For
0xece8d5b,weighted,"['For', 'Against']","[9238257.821819123, 0.0]",11729100.481171651,False,For


In [35]:
score_diffrences_dfs["Frax"]["outcome_changed"].value_counts(normalize=True)

False    0.98
True     0.02
Name: outcome_changed, dtype: float64

Whilst enourmous voting power was filtered out from all the proposals listed, just one had their outcomes changed (2%).

If we look at the proposal which attracted the most voting power, [this proposal](https://snapshot.org/#/frax.eth/proposal/0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a) to renew the grant for Flywheel for 2023 (in the amount of approx $214.2k, split 50/50 in FXS and FRAX) for the promotion of "the Frax ecosystem and producing high-qulity DeFi content", we see that just over 50% of the voting power was removed from this proposal after filtering out whales (approx 56.7%).

Though, because the opposition voting power for this proposal was a paltry 1E-5% of the vote, filtering out whales did not change the outcome. This clearly was not a controversial outcome to this proposal by number of voting participants (FXS token holders which participated in this propsal).

Indeed, Frax's [forum post](https://gov.frax.finance/t/fip-178-flywheel-grant-renewal-for-2023/2068) regarding this proposal had mostly positive things to say about Flywheel, which started out as a podcast promoting Frax in the wake of the collapse of Terra/Luna. Stating that since their last grant given to Flywheel in 2022, their social following has increased by 2770% on Twitter and 3140% on Youtube. They also serve as an educating branch for Frax, organizing meetups and also "attracted builders from across the ecosystem".

What's interesting about this proposal is that whale participation contributed to just over 56.7% of the voting power for this proposal. This is probably the most fairly represented outcome looked at so far, with the outcome being clearly popular with whales and the average voter.


In [16]:
propsal_choices = plutocracy_report_data['Frax'][plutocracy_report_data['Frax']['proposal_id'] == '0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Frax"].loc["0xece8d5b"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Frax'][plutocracy_report_data['Frax']['proposal_id'] == '0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,For,Against
Scores,1.172897e+07,131.88977
Score Differeces,9.238258e+06,0.00000


---
If we now look at Decentraland, we actually observe that there is quite a bad case of concentration of votes in the hands of whales.

In [17]:
score_diffrences_dfs["Decentraland"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0xdbe9b2d,single-choice,"['yes', 'no']","[4002000.0, 0]",4018284.304020843,False,yes
0x7a6a819,single-choice,"['yes', 'no']","[4260611.0495, 9364.50282589451]",4271209.566700251,False,yes
0x7acae7a,single-choice,"['yes', 'no']","[204322.76189336783, 4002000.0]",4207838.073517978,False,no
0x526eab1,single-choice,"['yes', 'no']","[624335.8827633569, 4677227.906455184]",5332049.831906232,True,yes
0x187559b,single-choice,"['yes', 'no']","[4013057.4165858557, 573884.5576664917]",4589827.856828686,False,yes
0x8b84875,single-choice,['yes I want someone to pick this up down the ...,"[1758.89825942416, 4004847.239929493, 312680.3...",4879182.104053254,True,yes I want someone to pick this up down the line
0x4913bba,single-choice,"['This is a good idea', 'This is a bad idea ',...","[6889278.390786966, 11246.553231335398, 6511.0...",7741409.420432587,False,This is a good idea
0x9b79389,single-choice,"[""We should diversify the DAO's treasury"", ""We...","[394684.30392451526, 4655.417903198702, 0]",400381.580846566,False,We should diversify the DAO's treasury
0x24be3be,single-choice,"['yes', 'no']","[4097189.096306793, 184549.89306984315]",4284151.947193912,False,yes


In [37]:
score_diffrences_dfs["Decentraland"]["outcome_changed"].value_counts(normalize=True)

False    0.76
True     0.24
Name: outcome_changed, dtype: float64

For example, this proposal to set a duration period for the tenure of Decentraland DAO commitee members saw 99.9% of voting power being concentrated to whales, with 94.85% of propsal voting power being allocated to voting for the proposal to not pass.

In [18]:
propsal_choices = plutocracy_report_data['Decentraland'][plutocracy_report_data['Decentraland']['proposal_id'] == '0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Decentraland"].loc["0x7f6fed8"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Decentraland'][plutocracy_report_data['Decentraland']['proposal_id'] == '0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,Yes,No,Invalid question/options
Scores,168338.243827,1.394610e+06,0.025206
Score Differeces,166209.638630,1.392896e+06,0.000000


Obviously a case of centralized power voting for their own self interests, its not suprising that if whales were factored out of the proposal (which would've given existing and future committee members set terms, making the roles more democratic) the proposal would've passed, albeit, with very low voting power in comparison.

---
Going back to DeFi, lets now look at Curve Finance.

In [19]:
score_diffrences_dfs["Curve Finance"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0x0eb23ea,single-choice,"['Yes', 'No']","[23462.994316173717, 511330.4724932205]",574553.874246682,True,Yes
0x43c689e,single-choice,"['Do Nothing', '2Pool + current 3Pool', '2Pool...","[0.0, 559204.7021513436, 2510541.702086272]",4272657.271177988,False,2Pool + current 3Pool + 4Pool
0xa11d4d1,single-choice,"['Yes', 'No']","[26054297.07744241, 0]",26097304.209488451,False,Yes
0x026a69e,single-choice,"['yes', 'no']","[1891.6266956075615, 0]",6893.077810757,False,yes
0x93cf3af,single-choice,"['sì', ' no']","[0.0, 653.7322378921115]",1141.553091741,True,sì
0xf29dd4b,single-choice,"['会', '不会']","[0, 1939.6283319658169]",3081.174372770,False,不会
0x3b2d049,single-choice,"['OUI', 'NON']","[472.8191613610968, 0]",487.813738095,False,OUI
0x4ebcc4c,single-choice,"['YES', 'NO']","[0.0, 472.81984655999344]",487.814445790,True,YES
0xeb4d67f,single-choice,"['yes', 'no']","[1945.3353636422128, 0]",2437.402576288,False,yes


In [39]:
score_diffrences_dfs["Curve Finance"]["outcome_changed"].value_counts(normalize=True)

False    0.714286
True     0.285714
Name: outcome_changed, dtype: float64

In [20]:
propsal_choices = plutocracy_report_data['Curve Finance'][plutocracy_report_data['Curve Finance']['proposal_id'] == '0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Curve Finance"].loc["0x0eb23ea"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Curve Finance'][plutocracy_report_data['Curve Finance']['proposal_id'] == '0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,Yes,No
Scores,45352.592228,529201.282019
Score Differeces,23462.994316,511330.472493


In [21]:
score_diffrences_dfs["Radicle"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0xf95b0e0,single-choice,"['For', 'Against']","[2000000.0, 0]",4133689.798082654,False,For
0x646e927,single-choice,"['Yes', 'No', 'Abstain']","[3730867.698436455, 0, 0]",6163639.615623794,False,Yes
bafkreia4,single-choice,"['For', 'Make No Change', 'Abstain']","[2654186.698436455, 0, 0]",6922974.673405429,False,For
0x16400b2,basic,"['For', 'Against', 'Abstain']","[3090365.0190418432, 0, 0]",4971478.525081047,False,For
0x704025e,basic,"['For', 'Against', 'Abstain']","[3090365.0190418432, 0, 0]",4946306.104718991,False,For
QmW64iqHQ,basic,"['For', 'Against', 'Abstain']","[478191.4276382547, 0, 0]",886529.953514751,False,For
0xe9158c0,single-choice,"['Yes', 'No', 'Abstain']","[478191.4276382547, 0, 0]",756591.962006816,False,Yes
0x4c289dd,single-choice,"['Yes', 'No', 'Abstain']","[22191.20317652479, 0, 0.0]",54157.179859683,True,No
QmepPgXwo,single-choice,"['Aye', 'Nay']","[20708.27072226024, 90218.6509325973]",201262.680259390,True,Aye


In [40]:
score_diffrences_dfs["Radicle"]["outcome_changed"].value_counts(normalize=True)

False    0.818182
True     0.181818
Name: outcome_changed, dtype: float64

In [23]:
score_diffrences_dfs["Compound"]

,type,choices,score_diffrences,total_vp,outcome_changed,outcome
Proposal ID,,,,,,
0xba069fb,single-choice,"['Yes', 'No']","[205387.6973835756, 0.0]",205482.098511252,False,Yes
bafkreiba,single-choice,"['Migrate to Commonwealth', 'Do Nothing']","[100044.20000815867, 0.0]",100096.776618262,False,Migrate to Commonwealth
bafkreiar,single-choice,"['5%', '4%', '3%', '2%', '1%', '0%', 'Abstain']","[151547.28029712665, 1.0000999999999998, 3.895...",151614.525233746,False,5%
bafkreibb,single-choice,"['Decrease RF for USDT & DAI', 'Increase RF fo...","[50015.58777338891, 151502.5046347422, 10.9855...",201588.163223679,True,Wait until Compound III


In [41]:
score_diffrences_dfs["Compound"]["outcome_changed"].value_counts(normalize=True)

False    0.75
True     0.25
Name: outcome_changed, dtype: float64